In [1]:
# Append web folder to system path
import sys
sys.path.append('../web')

In [2]:
from db_config import Base
from Database import Station, Availability, Weather, WeatherPredictive
from sqlalchemy import create_engine, func, Column, String, Integer, Double, Boolean, MetaData
from sqlalchemy.orm import sessionmaker
import json
import sys
import requests
from datetime import datetime, timedelta

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

In [4]:
# Get the db_info
with open('../dbinfo.json') as f:
    db_info = json.load(f)


USER = db_info['dbConnection']['USER']
PASSWORD = db_info['dbConnection']['PASSWORD']
URI = db_info['dbConnection']['URI']
PORT = db_info['dbConnection']['PORT']
DB = db_info['dbConnection']['DB']
WEATHER_API_KEY = db_info['weatherKey']
FORECAST_URI = 'https://api.openweathermap.org/data/2.5/forecast'


# Create a new session
engine = create_engine(
    'mysql+pymysql://{}:{}@localhost:{}/{}'.format(USER, PASSWORD, PORT, DB), echo=True)
Base.metadata.create_all(bind=engine)
Session = sessionmaker(bind=engine)
session = Session()
print("connected")

2024-04-03 19:52:02,031 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-04-03 19:52:02,032 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:52:02,060 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-04-03 19:52:02,061 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:52:02,076 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-04-03 19:52:02,078 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:52:02,106 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-03 19:52:02,108 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`stations`
2024-04-03 19:52:02,109 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:52:02,134 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`availability`
2024-04-03 19:52:02,135 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:52:02,147 INFO sqlalchemy.engine.Engine DESCRIBE `db_on_your_bike`.`weather`
2024-04-03 19:52:02,149 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:52:0

In [12]:
midnight = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

weather_historical = session.query(Weather).filter(
        Weather.time_updated > midnight).all()

# for row in weather_historical:
#     print(row.time_updated, row.wind_speed, row.temperature, row.humidity)

weather_historical_df = pd.DataFrame([row.__dict__ for row in weather_historical])
weather_historical_df = weather_historical_df[['time_updated', 'temperature', 'wind_speed', 'humidity', 'type']]
weather_historical_df.head()

2024-04-03 19:56:34,184 INFO sqlalchemy.engine.Engine SELECT weather.time_updated AS weather_time_updated, weather.type AS weather_type, weather.description AS weather_description, weather.temperature AS weather_temperature, weather.feels_like AS weather_feels_like, weather.min_temp AS weather_min_temp, weather.max_temp AS weather_max_temp, weather.humidity AS weather_humidity, weather.wind_speed AS weather_wind_speed, weather.visibility AS weather_visibility, weather.clouds AS weather_clouds, weather.sunrise AS weather_sunrise, weather.sunset AS weather_sunset 
FROM weather 
WHERE weather.time_updated > %(time_updated_1)s
2024-04-03 19:56:34,186 INFO sqlalchemy.engine.Engine [cached since 271.9s ago] {'time_updated_1': datetime.datetime(2024, 4, 3, 0, 0)}


,time_updated,temperature,wind_speed,humidity,type
0,2024-04-03 00:00:05,8.18,5.66,92,Rain
1,2024-04-03 00:30:04,8.18,6.17,91,Rain
2,2024-04-03 01:00:05,8.21,6.69,91,Rain
3,2024-04-03 01:30:05,8.13,6.17,92,Rain
4,2024-04-03 02:00:05,8.18,6.69,93,Rain


In [6]:
# Reflect the table
metadata = MetaData()
metadata.reflect(bind=engine)

# Access the table
table = metadata.tables['weather_predictive']
print(table.columns.keys())

2024-04-03 19:52:02,489 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-03 19:52:02,490 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `db_on_your_bike`
2024-04-03 19:52:02,490 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:52:02,516 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `availability`
2024-04-03 19:52:02,518 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:52:02,541 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `stations`
2024-04-03 19:52:02,542 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:52:02,560 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `weather`
2024-04-03 19:52:02,561 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:52:02,580 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `weather_predictive`
2024-04-03 19:52:02,581 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-03 19:52:02,601 INFO sqlalchemy.engine.Engine 
                    select table_schema, table_name, column_name
                    from informatio

In [7]:
# weather_historical = session.query(WeatherPredictive).all()

2024-04-03 19:52:02,675 INFO sqlalchemy.engine.Engine SELECT weather_predictive.forecast_time AS weather_predictive_forecast_time, weather_predictive.type AS weather_predictive_type, weather_predictive.description AS weather_predictive_description, weather_predictive.temperature AS weather_predictive_temperature, weather_predictive.feels_like AS weather_predictive_feels_like, weather_predictive.min_temp AS weather_predictive_min_temp, weather_predictive.max_temp AS weather_predictive_max_temp, weather_predictive.rain_3h AS weather_predictive_rain_3h, weather_predictive.humidity AS weather_predictive_humidity, weather_predictive.wind_speed AS weather_predictive_wind_speed, weather_predictive.clouds AS weather_predictive_clouds 
FROM weather_predictive
2024-04-03 19:52:02,679 INFO sqlalchemy.engine.Engine [generated in 0.00366s] {}


In [14]:
weather_historical_df = pd.DataFrame(
    [row.__dict__ for row in weather_historical])
weather_historical_df = weather_historical_df[[
    'time_updated', 'temperature', 'wind_speed', 'humidity', 'type']]

In [18]:
weather_preditive = session.query(WeatherPredictive).all()
weather_predictive_df = pd.DataFrame([row.__dict__ for row in weather_preditive])
weather_predictive_df['type'] = weather_predictive_df['weather_type']
weather_predictive_df = weather_predictive_df[[
    'time_updated', 'temperature', 'wind_speed', 'humidity', 'type']]
weather_predictive_df.head()



2024-04-03 19:58:47,599 INFO sqlalchemy.engine.Engine SELECT weather_predictive.forecast_time AS weather_predictive_forecast_time, weather_predictive.type AS weather_predictive_type, weather_predictive.description AS weather_predictive_description, weather_predictive.temperature AS weather_predictive_temperature, weather_predictive.feels_like AS weather_predictive_feels_like, weather_predictive.min_temp AS weather_predictive_min_temp, weather_predictive.max_temp AS weather_predictive_max_temp, weather_predictive.rain_3h AS weather_predictive_rain_3h, weather_predictive.humidity AS weather_predictive_humidity, weather_predictive.wind_speed AS weather_predictive_wind_speed, weather_predictive.clouds AS weather_predictive_clouds 
FROM weather_predictive
2024-04-03 19:58:47,615 INFO sqlalchemy.engine.Engine [cached since 404.9s ago] {}


,time_updated,temperature,wind_speed,humidity,type
0,2024-04-03 21:00:00,10.11,2.07,83,Rain
1,2024-04-04 00:00:00,8.73,1.48,89,Clouds
2,2024-04-04 03:00:00,6.45,1.20,94,Clouds
3,2024-04-04 06:00:00,6.78,0.99,95,Clouds
4,2024-04-04 09:00:00,9.62,1.47,87,Clouds


In [25]:
station_id = 2

# use pd.DataFrame because data is already an object
# predicted_weather_df = pd.DataFrame(weather_info['list'])
# predicted_weather_df['temperature'] = [row['main']['temp'] for row in weather_info['list']]
# predicted_weather_df['humidity'] = [row['main']['humidity'] for row in weather_info['list']]
# predicted_weather_df['wind_speed'] = [row['wind']['speed'] for row in weather_info['list']]
# predicted_weather_df['type'] = [row['weather'][0]['main']
#                                     for row in weather_info['list']]
# predicted_weather_df['time_updated'] = pd.to_datetime(predicted_weather_df['dt_txt'])

# predicted_weather_df['humidity'] = predicted_weather_df['humidity'].astype('int64')

# # Convert temperature and wind_speed to float64
# predicted_weather_df['temperature'] = predicted_weather_df['temperature'].astype('float64')
# predicted_weather_df['wind_speed'] = predicted_weather_df['wind_speed'].astype('float64')


# predicted_weather_df = predicted_weather_df[['time_updated', 'temperature', 'wind_speed', 'humidity', 'type']]

# predicted_weather_df.head()



In [19]:
weather_combined = pd.concat([weather_historical_df, weather_predictive_df])
weather_combined.head()

,time_updated,temperature,wind_speed,humidity,type
0,2024-04-03 00:00:05,8.18,5.66,92,Rain
1,2024-04-03 00:30:04,8.18,6.17,91,Rain
2,2024-04-03 01:00:05,8.21,6.69,91,Rain
3,2024-04-03 01:30:05,8.13,6.17,92,Rain
4,2024-04-03 02:00:05,8.18,6.69,93,Rain


In [20]:
current_date = datetime.now()

# Generate a list of hours for today
hours_today = [current_date.replace(hour=h, minute=30, second=0, microsecond=0) for h in range(24)]
hourly_df = pd.DataFrame(hours_today, columns=['time_updated'])

hourly_df.head()

,time_updated
0,2024-04-03 00:30:00
1,2024-04-03 01:30:00
2,2024-04-03 02:30:00
3,2024-04-03 03:30:00
4,2024-04-03 04:30:00


In [21]:
df = pd.merge_asof(hourly_df, weather_combined, on='time_updated')
df.head()

,time_updated,temperature,wind_speed,humidity,type
0,2024-04-03 00:30:00,8.18,5.66,92,Rain
1,2024-04-03 01:30:00,8.21,6.69,91,Rain
2,2024-04-03 02:30:00,8.18,6.69,93,Rain
3,2024-04-03 03:30:00,8.36,6.17,94,Clouds
4,2024-04-03 04:30:00,8.46,6.17,93,Rain


In [22]:
days = ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
        'Wednesday']

# One hot encode day of the week
df['weekday'] = df['time_updated'].dt.day_name()
df['hour'] = df['time_updated'].dt.hour
for day in days:
    df[day] = df['weekday'] == day



df['rain'] = df['type'] == 'Rain'

df.drop('time_updated', axis=1, inplace=True)
df.drop('weekday', axis=1, inplace=True)
df.drop('type', axis=1, inplace=True)

df = df[['temperature', 'wind_speed', 'humidity', 'hour', 'rain', 'Friday',
       'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']]

In [23]:
df

,temperature,wind_speed,humidity,hour,rain,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,8.18,5.66,92,0,True,False,False,False,False,False,False,True
1,8.21,6.69,91,1,True,False,False,False,False,False,False,True
2,8.18,6.69,93,2,True,False,False,False,False,False,False,True
3,8.36,6.17,94,3,False,False,False,False,False,False,False,True
4,8.46,6.17,93,4,True,False,False,False,False,False,False,True
5,8.46,7.72,94,5,True,False,False,False,False,False,False,True
6,8.05,6.69,95,6,True,False,False,False,False,False,False,True
7,7.50,7.20,93,7,False,False,False,False,False,False,False,True
8,7.37,7.20,93,8,False,False,False,False,False,False,False,True
9,7.53,7.72,94,9,False,False,False,False,False,False,False,True


In [26]:
with open(f'../ML_models/station_{station_id}.pkl', 'rb') as file:
    # Load the model from the file
    poly_reg_model = pickle.load(file)

poly = PolynomialFeatures(degree=3, include_bias=False)
poly_features = poly.fit_transform(df)

df['predicted_available'] = poly_reg_model.predict(poly_features)

In [27]:
df

,temperature,wind_speed,humidity,hour,rain,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,predicted_available
0,8.18,5.66,92,0,True,False,False,False,False,False,False,True,0.042450
1,8.21,6.69,91,1,True,False,False,False,False,False,False,True,1.008216
2,8.18,6.69,93,2,True,False,False,False,False,False,False,True,0.359445
3,8.36,6.17,94,3,False,False,False,False,False,False,False,True,8.298418
4,8.46,6.17,93,4,True,False,False,False,False,False,False,True,4.162119
5,8.46,7.72,94,5,True,False,False,False,False,False,False,True,2.920615
6,8.05,6.69,95,6,True,False,False,False,False,False,False,True,3.863218
7,7.50,7.20,93,7,False,False,False,False,False,False,False,True,11.260365
8,7.37,7.20,93,8,False,False,False,False,False,False,False,True,11.951043
9,7.53,7.72,94,9,False,False,False,False,False,False,False,True,12.420441
